In [1]:
# dendrite_revised
import numpy as np
import math 
import matplotlib.pyplot as plt
import random
import time
from scipy.spatial import cKDTree
start = time.time()
#time step in seconds
''' upto index 4 are ions '''
# Ionic Radii (in nanometers)
sizes= {"H+":0.053, "Mg2+":0.065, "NO3-":0.3, "H2PO4-":0.4,  "Mg(H2PO4)+":0.65, "Mg(NO3)2":0.6, "H3PO4":0.6, "HNO3":0.5}
molecular_matrix = {"Mg(NO3)2":0,"HNO3":0,"H3PO4":0}
p_asn = {"HNO3":0.132,"H3PO4":0.966,"Mg(NO3)2":0.316,"Mg(H2PO4)+":0.98} #porbability of association
size_mhpt = 0.25 #in nm

# sizes = np.array( [0.053, 0.065, 0.3, 0.4, 0.65, 0.6, 0.6, 0.5])
                   #H+ Mg2+ NO3- H2PO4-  Mg(H2PO4)+ Mg(NO3)2 H3PO4 HNO3
# Diffusion Coefficients in square meters per second (m²/s)

D_coeff = {"H+":9.31e-9, "Mg2+":7.5e-10, "NO3-":1.0e-9, "H2PO4-":8.67e-10,  "Mg(H2PO4)+":7.5e-10, "Mg(NO3)2":1.25e-9, "H3PO4":1.25e-9, "HNO3":1.1e-9}


# const = np.array([
#     9.31e-9,   # H+ (Hydrogen ion) in water at 25°C
#     7.5e-10,   # Mg2+ (Magnesium ion) in water at 25°C
#     1.0e-9,    # NO3- (Nitrate ion) in water at 25°C
#     8.67e-10,  # H2PO4- (Dihydrogen phosphate ion) in water at 25°C
#     7.5e-10,   # Mg(H2PO4)+ (Magnesium Dihydrogen Phosphate) in water at 25°C
#     1.25e-9,   # Mg(NO3)2 (Magnesium Nitrate) in water at 25°C
#     1.25e-9,   # H3PO4 (Phosphoric Acid) in water at 25°C
#     1.1e-9    # HNO3 (Nitric Acid) in water at 25°C
# ])

kb = 8.6173303 * 1e-5   #it's in ev/k
charge_dict = {"H+": 1, "Mg2+": 2, "NO3-": -1, "H2PO4-": -1, "Mg(H2PO4)+": 1, "Mg(NO3)2": 0, "H3PO4": 0, "HNO3": 0}
elementary_charge = 1.609 * 1e-19  # in C

q = {}
for species, charge in charge_dict.items():
    q[species] = charge * elementary_charge

T  = 298                #it's in K obviously!!

Mu = {}
for species, D in D_coeff.items():
    Mu[species] = q[species] * D / (kb * T)



del_t = 10 ** -6                                        # i.e., micro seconds
d_att =  0.238 # nm                                     # distance of attraction in nano metre         
d_att2 = (d_att) * 2  #  nm                             # cutoff distance
V_cathode = 0 # volts                                   # voltage of cathode and anode 
V_anode = 2 # volts

# ion_radius = 0.12    # nm                                 nanometre
# d2 = 1.3 * sizes                               # distance for smoothening the voltage
# D_coeff = 1.4 * 10 ** (-14) #metre ^2 / second          # diffusion coefficient of Li ion
# Mu = 5.6 * 10 ** (-13) #metre^2 /(Volt second)          # mobility of Li ion 
theta = {}
eeta = {}
for species, D in D_coeff.items():
    theta[species] = math.sqrt(2 * D * del_t)
    eeta[species] = Mu[species] * del_t   
La , Lb = 13.6 , 13.6                                   # domain size La x Lb in nano metres
#defining the system size
m = 50
n = 50
x = np.linspace(0, La, m)
y = np.linspace(0, Lb, n)
potential_ion_values= {}
xv , yv = np.meshgrid(x , y , indexing='xy')

for key in list(sizes.keys())[:4]:
    potential_values_ions = np.zeros((len(x),len(y)) , dtype = float)
    potential_ion_values[key]= potential_values_ions

potential_values_cat_i = {} #Mg() ion
potential_values_ions = np.zeros((len(x),len(y)) , dtype = float)
potential_values_cat_i["Mg(H2PO4+)"]= potential_values_ions


In [ ]:
potential_values = np.zeros((len(x),len(y)) , dtype = float)
for i in range(1,m-1):
    for j in range(1,n-1):
        b = potential_values
        b[0][:] = V_anode   #negative
        b[m-1][:] = V_cathode   #positive
        b[i][j] = V_anode + ((V_cathode - V_anode) * i/m)
        potential_values = b
#define potts_model(atom_matrix, ion_matrix)
def FDM_potentials():
    epsilon = 1e-8
    V_old = np.zeros((len(x), len(y)), dtype=float)
    V_new = potential_values

    # Set boundary conditions
    V_new[0, :] = V_cathode   # Positive
    V_new[(m-1), :] = V_anode    # Negative
    iterable1 = 0
    while (True):
        print(iterable1)
        iterable1 = iterable1+1
        V_old = V_new.copy()  # Store the current values in V_old
        for i in range(1, m-1):
            for j in range(0, n):
                V_new[i, j] = 0.25 * (V_old[i+1, j % n] +
                                      V_old[i-1, j % n] +
                                      V_old[i, (j + 1) % n] +
                                      V_old[i, (j - 1) % n])

        diff = np.amax(np.abs(np.round((V_new - V_old), decimals=9)))
        if diff < epsilon:
            break

    # print(V_new)
    # plt.imshow(V_new)
    # plt.colorbar()
    # plt.show()
    return V_new

def FDM_potentials_2_1(potentials,a_points,d2):#here potentials and a_points should correpsond to each atom
    epsilon = 1e-8
    # V_old = np.zeros((len(x), len(y)), dtype=float)
    # V_new = np.zeros((len(x), len(y)), dtype=float)
    V_new = potentials
    V_new[0, :] = V_cathode   # Positive
    V_new[(m-1), :] = V_anode    # Negative
    iterable2 = 0
    
    while (True):
        print(iterable2)
        iterable2 += 1            
        V_old = V_new.copy()  # Store the current values in V_old
        for i in range(1, m-1):
            for j in range(0, n):
                V_new[i, j] = 0.25 * (V_old[i+1, j % n] +
                                      V_old[i-1, j % n] +
                                      V_old[i, (j + 1) % n] +
                                      V_old[i, (j - 1) % n])

        diff = np.amax(np.abs(np.round((V_new - V_old), decimals=9)))
        for atom in a_points:
            x1, y1 = xv[atom[0], atom[1]], yv[atom[0], atom[1]]
            distances_squared = (xv - x1) ** 2 + (yv - y1) ** 2
            points_inside_circle_indices = np.where(distances_squared <= d2) #here what should d2 be? 
            V_new[points_inside_circle_indices] = V_anode
        # if iterable2%1000 == 0:
        #     print("difference:  ", diff)
        if (diff < epsilon) or iterable2==50:
            break
    # print(a_points)
    return V_new
 

In [ ]:
dl = round(xv[0][1] - xv[0][0] , 6)
def FDM_Electric_Field_Vector(b):
    E_i =np.zeros((len(x),len(y)) , dtype = float)
    E_j =np.zeros((len(x),len(y)) , dtype = float)
    b = np.array(b)
    for i in range(1,m-1):
        for j in range(1,n-1): 
            E_j[i, j] = (-(b[(i+m-1-1)%(m-1), (j+n-1)%(n-1)] - b[(i+m-1+1)%(m-1), (j+n-1)%(n-1)] ) / dl)
            E_i[i,j] = -(-(b[int((i+m-1)%(m-1)) , int((j+n-1-1)%(n-1))] - b[int((i+m-1)%(m-1)) , int((j+n-1+1)%(n-1))])/dl)  
    return E_i , E_j

def Find_the_index(target_xi,target_yi):
    i_index=0
    j_index=0
    i_index = abs(round((target_yi/dl)))%(n-1) 
    j_index = abs(round(target_xi/dl))%(m-1)
    return [i_index , j_index]

def potential_matrix_transform(potential_matrix):
    dummy_matrix = np.zeros((len(x),len(y)) , dtype = float)
    j = 0
    for i in range((m-1),-1,-1):
        dummy_matrix[j,:] = potential_matrix[i,:]
        j+=1
    return dummy_matrix


In [ ]:
ion_positions = []  # Collect all ion positions
for ion, ion_positions_list in position_ions.items():
    ion_positions.extend(ion_positions_list)

ion_kdtree = cKDTree(ion_positions)

def find_nearest_counterion(ion_position, ion_name):
    # Use the k-d tree to find the nearest counterion
    nearest_distance, nearest_index = ion_kdtree.query(ion_position, k=1)

    # Find the ion name from the index
    nearest_ion_name = None
    for ion, ion_positions_list in position_ions.items():
        if ion_positions_list[nearest_index] == ion_position:
            nearest_ion_name = ion
            break

    # Ensure that the found ion is not of the same type as the current ion
    if nearest_ion_name == ion_name:
        return None  # Same ion found, no nearest counterion
    else:
        return nearest_ion_name

# Modify the ising_model function to use find_nearest_counterion
def ising_model(position_ions, sizes, molecular_matrix, p_asn):
    visited_ions = []

    for ion, ion_positions in position_ions.items():
        for ion_position in ion_positions:
            if ion_position in visited_ions:
                continue
            else:
                nearest_counterion_name = find_nearest_counterion(ion_position, ion)
                if nearest_counterion_name:
                    combination = ion + nearest_counterion_name
                    if combination not in visited_ions:
                        random_number = random.random()
                        if random_number < p_asn[combination]:
                            # Update the ion's position and molecular matrix
                            update_ion_position_and_molecular_matrix(ion, nearest_counterion_name)
                            visited_ions.append(combination)
    return position_ions, molecular_matrix

# You should also update the k-d tree after changing ion positions
def update_ion_positions(ion_name, new_position,position_ions):
    ion_kdtree = cKDTree(new_positions)
    position_ions[ion_name] = new_positions

# Example usage of updating ion positions:
new_ion_positions = position_ions["H+"]  # Example: replace with new positions
update_ion_positions("H+", new_ion_positions)

In [19]:
positions_ions = {}
#initializing random ions
sizes= {"H+":0.053, "Mg2+":0.065, "NO3-":0.3, "H2PO4-":0.4,  "Mg(H2PO4)+":0.65, "Mg(NO3)2":0.6, "H3PO4":0.6, "HNO3":0.5}
initial_ions = {"H+":10, "Mg2+":10, "NO3-":20, "H2PO4-":10,  "Mg(H2PO4)+":0}
updated_positions_ions = {}
for key,ion_num in initial_ions.items():
    positions_ions_element = []
    for i in range(0,ion_num):
        positions_ions_element.append([random.random() * La , La])
    positions_ions[key] = positions_ions_element
    updated_positions_ions[key]=np.zeros((ion_num,2))










no_of_base_atoms = 27                           #creating a base layer in anode i.e., the bottom layer of system
positions_base = np.linspace(0,La,no_of_base_atoms)
atom_matrix = np.zeros((2,no_of_base_atoms))
for i in range(no_of_base_atoms):
    atom_matrix[0][i]=positions_base[i]
    
atom_matrix = np.transpose(atom_matrix)
dup_atom_matrix = atom_matrix
t=0
img1 = 0                          # img variable to pinpoint the image name from the array 
img2 = 0
num_of_atom_added = 0            # number of atom added to count how much of total atom added
E_indices=[]
dendrite_height=[]
time_array=[]
dend = 0
print("\nTHE SIMULATION HAS STARTED... ")
num_ion_deleted=0



THE SIMULATION HAS STARTED... 


In [ ]:
#we have to sperately define the atom positions of non-ionic species present in the setup
#till now potentials is not found necessary to be assigned to each ionic species
#instead it's important that the ionic species are made to update with it's native eeta and theta values
#the only atoms to form dendrite is Mg(H2PO4)+

while (True):  
    print("\nEntering while loop")
    filename1 = "Cont_Sim_20___"+str(img1)+"__.jpg"
    filename2 = "/Users/ajithbharathvaaj/dendrite_21_test/Only_atom_add_21_"+str(img2)+"__.jpg"
    t = t + del_t
    E_indices = {}
    for key,values in positions_ions:
        for index,ions in enumerate(values):
            E_indices[key].append(Find_the_index(ions[0],ions[1])) 
    #so E_indices also is a dictionary now
    #
           
    if (t == del_t):
        potentials = FDM_potentials()
        E_x , E_y = FDM_Electric_Field_Vector(potentials)
        E_x_t , E_y_t = potential_matrix_transform(E_x) , potential_matrix_transform(E_y)
    p=0

    for key in positions_ions:
        for p ,(ions,E_idx) in enumerate(zip(positions_ions[key],E_indices[key]),start=0):
            p = p % len(updated_positions_ions[key])
            a_vector = random.uniform(-1, 1) 
            b_vector = random.uniform(-1,1)  
            mod_of_a_and_b = math.sqrt(a_vector **2 + b_vector **2)
            # g is a normalized 2 D vector
            g = np.array([a_vector/mod_of_a_and_b , b_vector/mod_of_a_and_b])
            A = [g_vector * theta[key] for g_vector in g]
            B = [eeta[key] * E_x_t[E_idx[0]][E_idx[1]] , eeta[key] * E_y_t[E_idx[0]][E_idx[1]]]
            A =np.array(A)
            B =np.array(B)
            print('A = ',A,'   B = ',B,'   ions = ',ions)
            updated_positions_ions[key][p] =(A + B +  np.round(np.array((ions)) * (10 ** -9) , decimals=20)  )/10**-9
            if (updated_positions_ions[key][p][1] > 13.6):
                updated_positions_ions[key][p][1] = 13.6
            if (updated_positions_ions[key][p][1] < 0):
                updated_positions_ions[key][p][1] = 0 
            if (updated_positions_ions[key][p][0] < 0) or (updated_positions_ions[key][p][0] > 13.6):
                updated_positions_ions[key][p][0] = (updated_positions_ions[key][p][0]) % 13.6

    
    
    
    #here atom matrix will only contain  the updated positions of position_ions["Mg(H2PO4)+"] as it will be
    #the only one to form the dendrites
    
    
    
    positions_ions = updated_positions_ions 

    ising_model(positions_ions,sizes,molecular_matrix,p_asn) 
    # Build a k-d tree for atoms
    atom_kdtree = cKDTree(atom_matrix) #atom matrix will only contain intermediate cation
    # Find the nearest neighbor within the cutoff distance for each ion
    ion_neighbors = atom_kdtree.query_ball_point(positions_ions, d_att2)
    ion_to_atom_transfer=[]
    for i, ion_position in enumerate(positions_ions["Mg(H2PO4)+"]):
        neighbor_indices = ion_neighbors[i]
        if neighbor_indices:
            distances_and_indices = [(np.linalg.norm(ion_position - atom_matrix[j]), i, j) for j in neighbor_indices]
            distances_and_indices.sort(key=lambda x: x[0])  # Sort by distance
            
            # Filter neighbors with the same minimum distance
            min_distance = distances_and_indices[0][0]
            min_distance_neighbors = [x for x in distances_and_indices if x[0] == min_distance]
            
            # Randomly choose one neighbor from those with the same minimum distance
            nearest_neighbor_distance, ion_index, atom_index = random.choice(min_distance_neighbors)
            ion_to_atom_transfer.append([ion_index, atom_index, nearest_neighbor_distance])
    
    if len(ion_to_atom_transfer)>0:
        print(ion_to_atom_transfer)
        print(np.shape(ion_to_atom_transfer))
        for neighbor in enumerate(ion_to_atom_transfer,start=0):
            ion_idx = neighbor[1][0]
            atom_idx = neighbor[1][1]
            distance = neighbor[1][2]
            normalize_x = (positions_ions["Mg(H2PO4)+"][ion_idx][0] - atom_matrix[atom_idx][0]) / distance
            normalize_y = (positions_ions["Mg(H2PO4)+"][ion_idx][1] - atom_matrix[atom_idx][1]) / distance
            new_atom_array = [ (atom_matrix[atom_idx][0] + normalize_x * d_att)%13.6 , (atom_matrix[atom_idx][1] + normalize_y * d_att)%13.6 ]
            #deleting from the positions_ions["Mg(H2PO4)+"] list 
            print("the previous ion position", positions_ions["Mg(H2PO4)+"][ion_idx])
            print("\nnearest atom position",atom_matrix[atom_idx])
            positions_ions["Mg(H2PO4)+"] = positions_ions["Mg(H2PO4)+"].tolist()  
            del positions_ions["Mg(H2PO4)+"][ion_idx%len(positions_ions["Mg(H2PO4)+"])]
            positions_ions["Mg(H2PO4)+"] = np.array(positions_ions["Mg(H2PO4)+"])
            # adding to the atom list
            
            
            atom_matrix = atom_matrix.tolist()
            atom_matrix.append(new_atom_array)
            atom_matrix = np.array(atom_matrix)
    # if len(ion_to_atom_transfer)>0:
    if len(ion_to_atom_transfer):
        num_ion_deleted = len(ion_to_atom_transfer)
        print(num_ion_deleted)
        for _ in range(0,num_ion_deleted):
            positions_ions["Mg(H2PO4)+"] = positions_ions["Mg(H2PO4)+"].tolist()
            positions_ions["Mg(H2PO4)+"].append([random.random() * La , La]) 
            positions_ions["Mg(H2PO4)+"] = np.array(positions_ions["Mg(H2PO4)+"]) 
        #  after this we have to assign the new atom positions into V_anode, and smoothen it by including many points
        # the smoothening is done in the FDM_potentials2() function itself.
        mask = np.isin(atom_matrix, dup_atom_matrix).all(axis=1)
        atom_matrix = atom_matrix[~mask]

        points_atom= []
        for q in atom_matrix:
            new_array = Find_the_index(q[0],abs(q[1] - La)) 
            points_atom.append(new_array)

        # below two lines calculates the electric potentials by solving laplace equation and  subsequently electric field vector
        t1 = time.time()
        print("\n potential is being calculated.............")
        potentials = np.array(potentials,dtype=float)
        potentials = FDM_potentials_2_1(points_atom)
        E_x , E_y = FDM_Electric_Field_Vector(potentials)
        t2 = time.time()
        print("time of completion of one potential and one E vector calc: ", t2-t1) 
        for _ in enumerate(atom_matrix,start=0):
            dendrite_y_axis  = [element[1] for element in atom_matrix]





            
        # to plot the figure and save it each and every 5 x atoms
        plt.figure(figsize=(8.5,8.5))
        ax = plt.axes()
        a_mat_combined = np.vstack((dup_atom_matrix,atom_matrix))
        atom_matrix_T = np.transpose(a_mat_combined)
        ax.scatter(atom_matrix_T[0][:],atom_matrix_T[1][:],c='#39FF14',s=45)
        #only transpose positions_ions array while plotting
        positions_ions_T = np.transpose(positions_ions) 
        ax.scatter(positions_ions_T[0,:],positions_ions_T[1,:],c='red',s=50)
        potentials_Mirror_Transform = potential_matrix_transform(potentials)
        ax.contour(xv,yv,potentials_Mirror_Transform,levels = 12,colors = 'gray',linestyles='None',linewidths=1) 
        ssf = 3 #subsample_factor
        magnitude = np.sqrt(E_x ** 2 + E_y ** 2)
        Ex_norm = E_x / np.amax(magnitude)
        Ey_norm = E_y / np.amax(magnitude) 
        E_x_t , E_y_t = potential_matrix_transform(Ex_norm), potential_matrix_transform(Ey_norm)
        ax.quiver(xv[::ssf, ::ssf],  # Subsampled X-coordinates
                    yv[::ssf, ::ssf],  # Subsampled Y-coordinates
                    E_x_t[::ssf, ::ssf],  # Subsampled X-components
                    E_y_t[::ssf, ::ssf],  # Subsampled Y-components
                    scale=0.8 , color='blue',pivot='tip')  # Adjust the arrow length
        ax.set_xlabel('CGMC DENDRITE SIMULATION \nCapture Time: '+ str(round(t,8))+" seconds \n max height of dendrite = "+str(abs(max(dendrite_y_axis)))+" nm", loc = 'center',fontsize=14,fontweight='bold',fontname= 'Times New Roman' )
        ax.set_xlim(0,13.6)
        ax.set_ylim(0,13.6)
        plt.savefig(filename2,dpi = 500)
        # print("the image is under the name",filename2)
        plt.close()
        img2 = img2 + 1
        atom_matrix = a_mat_combined
       
    # now to check if the length of the dendrite exceeds the La - 0.1
    for _ in enumerate(atom_matrix,start=0):
        dendrite_y_axis  = [element[1] for element in atom_matrix]
    dendrite_height.append(abs(max(dendrite_y_axis)))
    time_array.append(t)
    dend = dend+1
    for _ in enumerate(positions_ions,start=0):
        ion_y_axis = [element[1] for element in positions_ions]
    # if the maximum of the dendrite is greater than or equal to  La - 0.4, then stop the simulation
    # if max(dendrite_y_axis) >= 1:
    if abs(max(dendrite_y_axis)) >=(La-0.1):
        print("the sim ended because the dendrite length has exceeded the system ")
        plt.figure(figsize=(8.5,8.5))
        ax1 = plt.axes()
        ax1.plot(time_array,dendrite_height,c='blue')
        ax1.set_xlabel('Time (seconds)',loc = 'center',fontsize=16,fontweight='bold',fontname= 'Times New Roman' )
        ax1.set_ylabel('Dendrite height (nm)',loc = 'center',fontsize=16,fontweight='bold',fontname= 'Times New Roman' ) 
        ax.set_xlim(min(time_array),max(time_array))
        ax.set_ylim(min(dendrite_height),max(dendrite_height))
        plt.savefig("dendrite height vs time.jpg",dpi = 600)
        plt.close()
        break


end = time.time()
print("\n \n The total time of computation is : ",(end-start)/60," minutes    ",(end-start)/3600," hours") 
print("YAYYYYY SIMULATION COMPLETED")